This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/robot.html).  I recommend having both windows open, side-by-side!


In [1]:
import os
import tempfile

from pydrake.all import ModelVisualizer, RigidTransform, StartMeshcat
from scene_synthesizer import BoxAsset
from scene_synthesizer.procedural_scenes import kitchen

from manipulation.remotes import AddRby1Remote

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()
tmpdir = tempfile.TemporaryDirectory()

In [ ]:
def VisualizeKitchen(meshcat):
    visualizer = ModelVisualizer(meshcat=meshcat)

    # Generate a random kitchen.
    scene = kitchen(seed=None)

    # Use scene_synthesizer to figure out where to place the robot in the scene.
    rby1_bbox = BoxAsset(extents=[0.662, 0.58, 1.47])
    scene.add_object(
        rby1_bbox,
        "rby1_bbox",
        connect_parent_id="refrigerator",
        connect_parent_anchor=("center", "front", "bottom"),
        connect_obj_anchor=("center", "back", "bottom"),
        translation=[0, -0.02, 0],
    )
    X_WRby1 = RigidTransform(scene.get_transform("rby1_bbox")) @ RigidTransform(
        [0, 0, -1.47 / 2]
    )
    scene.remove_object("rby1_bbox")

    # Export the kitchen to a urdf and add it to the visualizer.
    kitchen_urdf_path = os.path.join(tmpdir.name, "kitchen.urdf")
    scene.export(kitchen_urdf_path)
    visualizer.AddModels(kitchen_urdf_path)

    # Add the robot.
    AddRby1Remote(visualizer.parser().package_map())
    rby1_id = visualizer.parser().AddModels(url="package://rby1/urdf/model.drake.urdf")[
        0
    ]
    plant = visualizer.parser().plant()
    plant.SetDefaultFreeBodyPose(plant.GetBodyByName("base", rby1_id), X_WRby1)
    visualizer.Run(loop_once=True)


VisualizeKitchen(meshcat)